# Example: Neural Networks – Running bound tightening in parallel

In [8]:
using Distributed
using Gurobi
using Gogeta
using JuMP
using Random
using Flux

<span style="color: red;">In this example, we want to show how neural networks can be more quickly formulated as MILP using parallel computing. As in the previous example, we initialize the arbitrary neural network with random weights (the model is exactly the same as in the example about NN).</span>

In this example, we want to show how the neural networks can be faster formulated as MILP formulation using parallel computing. As in the previous example, we initialize the arbitrary neural network with random weights (the model in exactly the same as in the example about NN).

In [6]:
begin
    Random.seed!(1234);

    model = Chain(
        Dense(2 => 10, relu),
        Dense(10 => 50, relu),
        Dense(50 => 20, relu),
        Dense(20 => 5, relu),
        Dense(5 => 1)
    )
end

Chain(
  Dense(2 => 10, relu),                 # 30 parameters
  Dense(10 => 50, relu),                # 550 parameters
  Dense(50 => 20, relu),                # 1_020 parameters
  Dense(20 => 5, relu),                 # 105 parameters
  Dense(5 => 1),                        # 6 parameters
)                   # Total: 10 arrays, 1_711 parameters, 7.309 KiB.

Using `addprocs()`, we inntiailize 4 parallel processes or 'workers'.

In [15]:
addprocs(4)

4-element Vector{Int64}:
 22
 23
 24
 25

In order to prevent Gurobi from obtaining a new licence for each 'worker', we need to specify the same `Gurobi` environment for each one

In [ ]:
@everywhere using Gurobi
@everywhere ENV = Ref{Gurobi.Env}()

@everywhere function init_env()
    global ENV
    ENV[] = Gurobi.Env()
end

for worker in workers()
    fetch(@spawnat worker init_env())
end

Regardless of the solver, we must also specify `Gurobi` as optimizer.

In [ ]:
@everywhere using JuMP
@everywhere function set_solver!(jump)
    set_optimizer(jump, () -> Gurobi.Optimizer(ENV[]))
    set_silent(jump)
end

As before, we need to define boundaries for innitial variables in which MILP formulation gurantees the same output as neural network

In [ ]:
init_U = [1.0, 1.0];
init_L = [-1.0, -1.0];

Once the workers are set up, you can use  `NN_formulate!()` function with a parameter `parallel=true` 

In [ ]:
@everywhere using Gogeta
jump = Model()
@time U, L = NN_formulate!(jump, model, init_U, init_L; bound_tightening="standard", silent=true, parallel=true);

The function will again update constraints of the empty jump model and output the boundaries for the neurons. You can also change `bound_tightening` parameter to other approaches. Once you got this formulation, the model can be optmized in the same way as before.

In [ ]:
output_neuron = jump_model[:x][maximum(keys(jump_model[:x].data))]
@objective(jump_model, Max, output_neuron)
optimize!(jump_model)

println("The model found next solution:\n", value.(jump_model[:x][0, :]))
println("With objective function: ", objective_value(jump_model) )
solution = Float32.([i for i in value.(jump_model[:x][0, :])])
println("The output of the NN for solution given by jump model: ", NN_model(solution)[1])